# Sora2 Watermark Remover - Simple Version

シンプルな6ステップでSora動画からウォーターマークを除去します。

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. 依存関係のインストール

In [ ]:
%pip install -q "iopaint==1.2.2" "transformers==4.42.4" opencv-python loguru tqdm
!apt-get -qq update && apt-get -qq install -y ffmpeg

## 2. LaMaモデルのダウンロード

In [ ]:
!iopaint download --model lama

## 3. 最新コードの取得

In [ ]:
!curl -sL https://github.com/fulfulggg/Sora2WatermarkRemover/archive/refs/heads/main.tar.gz | tar xz --strip-components=1
print("\n✅ リポジトリのダウンロードが完了しました。")

## 4. 動画のアップロード

In [ ]:
from google.colab import files

print("動画ファイルをアップロードしてください:")
uploaded = files.upload()

input_file = list(uploaded.keys())[0]
# 絶対パスに正規化
import os, shutil
src = os.path.abspath(input_file)
_, ext = os.path.splitext(src)
input_path = f"/content/input_uploaded{ext or '.mp4'}"
if src != input_path:
    shutil.move(src, input_path)
print(f"\n✅ アップロード完了: {input_path}")

## 5. ウォーターマークの除去

Florence-2 AIがウォーターマークを自動検出して除去します。

In [ ]:
!python remwm.py "{input_path}" "/content/out.mp4" --overwrite

## 6. 結果のダウンロード

In [ ]:
print("結果をダウンロード中...")
files.download('/content/out.mp4')
print("\n✅ 完了！")